In [1]:
import pandas as pd
import numpy as np
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import gc
import h5py

from PIL import Image
from pyts.image import RecurrencePlot, GramianAngularField
from pyts.datasets import load_gunpoint
from pyts.image import MarkovTransitionField
from pyts.image import GramianAngularField

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
import tensorflow.python.keras.backend as K

from keras import optimizers
from keras import regularizers
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Flatten, Conv3D,Conv2D, MaxPooling3D, MaxPooling2D, Dropout, LSTM
from keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint
from keras.applications import VGG16
from keras.optimizers import Adam

Using TensorFlow backend.


# Make LSTM Data

In [2]:
def make_input_size(data):
    #data = data*255
    img = Image.fromarray(data[0].astype('uint8'), 'L')
    img = img.resize((224,224))
    img_arr = np.asarray(img)
    result_arr = img_arr
    
    for i in range(1,16):
        img = Image.fromarray(data[i].astype('uint8'), 'L')
        img = img.resize((224,224))
        img_arr = np.asarray(img)
        result_arr = np.concatenate([result_arr, img_arr], axis=0)
    
    result_arr = result_arr.reshape(-1,224,224)
    return result_arr

In [3]:
x_data = []
y_data = []
path = 'E:/Data/new_EEG/7680/'
file_list = os.listdir('input')

for file in file_list:
    data = pd.read_csv('input/'+file, header=0)
    for k in range(16):
        x_data.append(data.iloc[:,k])
    
    if file[:3] == 'sch':
        for a in range(16):
            y_data.append(1)
    else:
        for a in range(16):
            y_data.append(0)

x_data = np.array(x_data)
x_data = x_data.reshape(-1,7680,1)
y_data = np.array(y_data)

In [1]:
# Learning Curve

def draw_plot(hist):
    fig, loss_ax = plt.subplots()

    acc_ax = loss_ax.twinx()

    loss_ax.plot(hist.history['loss'], 'y', label='train loss')
    loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')

    acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
    acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')

    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    acc_ax.set_ylabel('accuray')

    loss_ax.legend(loc='upper left')
    acc_ax.legend(loc='lower left')

    plt.show()

# Model

In [6]:
def create_model():
   
    model = Sequential()
    model.add(LSTM(256, input_shape=(7680, 1)))#, return_sequences=True))
    #model.add(LSTM(128))
    #model.add(Dropout(rate=0.2))
    #model.add(Dense(2048, activation='relu'))
    #model.add(Dropout(0.5))
    #model.add(Dense(1024, activation='relu'))
    #model.add(Dropout(0.5))
    #model.add(Dense(256, activation='relu'))
    #model.add(Dense(64, activation='relu'))
    #model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
    
    #return model
    return model, model.get_weights()

In [7]:
kfold = KFold(n_splits=10, shuffle=True, random_state=1004)

In [8]:
def init_weight(same_old_model, first_weights):
    same_old_model.set_weights(weights)

In [ ]:
def cf_mat(test):
    y_pred = model.predict_classes(x_data[test])
    return confusion_matrix(y_data[test], y_pred)

In [9]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=50, restore_best_weights=True)

# Train & Test

In [10]:
num = 0
k_accuracy = list()
accuracy = list()

model, weights = create_model()
for train, test in kfold.split(x_data):
    #cuda.select_device (0)
    #model = create_model()
    num = num+1
    print("---------------------------epoch : ",num,"---------------------------")
    init_weight(model, weights)
    model.fit(x_data[train], y_data[train], epochs=50, batch_size=20, callbacks=[es])
    accuracy =  (model.evaluate(x_data[test], y_data[test]))
    k_accuracy.append(accuracy)
    
    mat.append(cf_mat(test))

---------------------------epoch :  1 ---------------------------
Epoch 1/50
1209/1209 [==============================] - 330s 273ms/step - loss: 0.7004 - accuracy: 0.5327
Epoch 2/50
1209/1209 [==============================] - 342s 283ms/step - loss: 0.6773 - accuracy: 0.5691
Epoch 3/50
1209/1209 [==============================] - 429s 355ms/step - loss: 0.6587 - accuracy: 0.5897
Epoch 4/50
1209/1209 [==============================] - 419s 347ms/step - loss: 0.6532 - accuracy: 0.5988
Epoch 5/50
1209/1209 [==============================] - 399s 330ms/step - loss: 0.6494 - accuracy: 0.5980
Epoch 6/50
1209/1209 [==============================] - 302s 250ms/step - loss: 0.6370 - accuracy: 0.6245
Epoch 7/50
1209/1209 [==============================] - 301s 249ms/step - loss: 0.6235 - accuracy: 0.6328
Epoch 8/50
1209/1209 [==============================] - 301s 249ms/step - loss: 0.6076 - accuracy: 0.6510
Epoch 9/50
1209/1209 [==============================] - 301s 249ms/step - loss: 0.6075

Epoch 26/50
1209/1209 [==============================] - 302s 250ms/step - loss: 0.3303 - accuracy: 0.8544
Epoch 27/50
1209/1209 [==============================] - 301s 249ms/step - loss: 0.3306 - accuracy: 0.8503
Epoch 28/50
1209/1209 [==============================] - 302s 250ms/step - loss: 0.3100 - accuracy: 0.8577
Epoch 29/50
1209/1209 [==============================] - 302s 250ms/step - loss: 0.3148 - accuracy: 0.8577
Epoch 30/50
1209/1209 [==============================] - 302s 250ms/step - loss: 0.2867 - accuracy: 0.8685
Epoch 31/50
1209/1209 [==============================] - 302s 249ms/step - loss: 0.2623 - accuracy: 0.8867
Epoch 32/50
1209/1209 [==============================] - 301s 249ms/step - loss: 0.2442 - accuracy: 0.8933
Epoch 33/50
1209/1209 [==============================] - 303s 250ms/step - loss: 0.2179 - accuracy: 0.9115
Epoch 34/50
1209/1209 [==============================] - 303s 251ms/step - loss: 0.2045 - accuracy: 0.9140
Epoch 35/50
1209/1209 [==============

---------------------------epoch :  4 ---------------------------
Epoch 1/50
1209/1209 [==============================] - 301s 249ms/step - loss: 0.7001 - accuracy: 0.5219
Epoch 2/50
1209/1209 [==============================] - 301s 249ms/step - loss: 0.6815 - accuracy: 0.5451
Epoch 3/50
1209/1209 [==============================] - 301s 249ms/step - loss: 0.6663 - accuracy: 0.6013
Epoch 4/50
1209/1209 [==============================] - 301s 249ms/step - loss: 0.6793 - accuracy: 0.5608
Epoch 5/50
1209/1209 [==============================] - 301s 249ms/step - loss: 0.6557 - accuracy: 0.6030
Epoch 6/50
1209/1209 [==============================] - 301s 249ms/step - loss: 0.6502 - accuracy: 0.6129
Epoch 7/50
1209/1209 [==============================] - 301s 249ms/step - loss: 0.6531 - accuracy: 0.5831
Epoch 8/50
1209/1209 [==============================] - 301s 249ms/step - loss: 0.6412 - accuracy: 0.6162
Epoch 9/50
1209/1209 [==============================] - 301s 249ms/step - loss: 0.6380

Epoch 26/50
1210/1210 [==============================] - 309s 255ms/step - loss: 0.3479 - accuracy: 0.8438
Epoch 27/50
1210/1210 [==============================] - 316s 261ms/step - loss: 0.3305 - accuracy: 0.8488
Epoch 28/50
1210/1210 [==============================] - 318s 263ms/step - loss: 0.3210 - accuracy: 0.8479
Epoch 29/50
1210/1210 [==============================] - 319s 263ms/step - loss: 0.2980 - accuracy: 0.8686
Epoch 30/50
1210/1210 [==============================] - 319s 264ms/step - loss: 0.2777 - accuracy: 0.8653
Epoch 31/50
1210/1210 [==============================] - 320s 264ms/step - loss: 0.2756 - accuracy: 0.8851
Epoch 32/50
1210/1210 [==============================] - 321s 265ms/step - loss: 0.2728 - accuracy: 0.8752
Epoch 33/50
1210/1210 [==============================] - 328s 271ms/step - loss: 0.2573 - accuracy: 0.8975
Epoch 34/50
1210/1210 [==============================] - 306s 253ms/step - loss: 0.2256 - accuracy: 0.9050
Epoch 35/50
1210/1210 [==============

---------------------------epoch :  7 ---------------------------
Epoch 1/50
1210/1210 [==============================] - 302s 249ms/step - loss: 0.7014 - accuracy: 0.5264
Epoch 2/50
1210/1210 [==============================] - 300s 248ms/step - loss: 0.6828 - accuracy: 0.5504
Epoch 3/50
1210/1210 [==============================] - 301s 249ms/step - loss: 0.6795 - accuracy: 0.5744
Epoch 4/50
1210/1210 [==============================] - 303s 250ms/step - loss: 0.6611 - accuracy: 0.5901
Epoch 5/50
1210/1210 [==============================] - 302s 249ms/step - loss: 0.6563 - accuracy: 0.5983
Epoch 6/50
1210/1210 [==============================] - 301s 249ms/step - loss: 0.6526 - accuracy: 0.5860
Epoch 7/50
1210/1210 [==============================] - 301s 249ms/step - loss: 0.6394 - accuracy: 0.6314
Epoch 8/50
1210/1210 [==============================] - 301s 249ms/step - loss: 0.6282 - accuracy: 0.6397
Epoch 9/50
1210/1210 [==============================] - 301s 249ms/step - loss: 0.6237

Epoch 26/50
1210/1210 [==============================] - 302s 250ms/step - loss: 0.3219 - accuracy: 0.8504
Epoch 27/50
1210/1210 [==============================] - 301s 249ms/step - loss: 0.3244 - accuracy: 0.8537
Epoch 28/50
1210/1210 [==============================] - 301s 249ms/step - loss: 0.2751 - accuracy: 0.8893
Epoch 29/50
1210/1210 [==============================] - 302s 249ms/step - loss: 0.2467 - accuracy: 0.9058
Epoch 30/50
1210/1210 [==============================] - 301s 249ms/step - loss: 0.2257 - accuracy: 0.9149
Epoch 31/50
1210/1210 [==============================] - 301s 249ms/step - loss: 0.2114 - accuracy: 0.9132
Epoch 32/50
1210/1210 [==============================] - 302s 250ms/step - loss: 0.1969 - accuracy: 0.9273
Epoch 33/50
1210/1210 [==============================] - 302s 249ms/step - loss: 0.1930 - accuracy: 0.9281
Epoch 34/50
1210/1210 [==============================] - 303s 251ms/step - loss: 0.2020 - accuracy: 0.9182
Epoch 35/50
1210/1210 [==============

---------------------------epoch :  10 ---------------------------
Epoch 1/50
1210/1210 [==============================] - 302s 249ms/step - loss: 0.6945 - accuracy: 0.5488
Epoch 2/50
1210/1210 [==============================] - 302s 249ms/step - loss: 0.6808 - accuracy: 0.5661
Epoch 3/50
1210/1210 [==============================] - 302s 250ms/step - loss: 0.6674 - accuracy: 0.5860
Epoch 4/50
1210/1210 [==============================] - 301s 249ms/step - loss: 0.6671 - accuracy: 0.5810
Epoch 5/50
1210/1210 [==============================] - 302s 249ms/step - loss: 0.6620 - accuracy: 0.5942
Epoch 6/50
1210/1210 [==============================] - 302s 250ms/step - loss: 0.6515 - accuracy: 0.6099
Epoch 7/50
1210/1210 [==============================] - 301s 249ms/step - loss: 0.6457 - accuracy: 0.6091
Epoch 8/50
1210/1210 [==============================] - 301s 249ms/step - loss: 0.6388 - accuracy: 0.6306
Epoch 9/50
1210/1210 [==============================] - 302s 249ms/step - loss: 0.635

In [12]:
k_accuracy

[[1.9035675622798778, 0.5407407283782959],
 [1.5002587110907943, 0.585185170173645],
 [1.4191483947965833, 0.585185170173645],
 [1.827363117646288, 0.5481481552124023],
 [1.226927662963298, 0.5970149040222168],
 [1.464383638616818, 0.5671641826629639],
 [1.2475048668348967, 0.60447758436203],
 [1.4554610946285191, 0.5746268630027771],
 [1.7687229536362548, 0.48507463932037354],
 [1.0314137330695765, 0.641791045665741]]

In [ ]:
mat